In [1]:
import httpx
import json

In [10]:
resp = httpx.get("https://files.jgi.doe.gov/search/?q=extorquens&a=false&h=false&d=asc&p=1&x=10&t=advanced&api_version=2")

In [2]:
# query = "metadata.sequencing_project.scientific_program_name:Microbial"
query = "1269263"
search_url = f"https://files.jgi.doe.gov/search/?q={query}&a=false&h=false&d=asc&p=1&x=10&t=simple&api_version=2"

# search_url = "https://files.jgi.doe.gov/img_file_list/?api_version=2&d=asc&q=microbial&of=%7B%22dataset_type%22%3A%5B%22Draft+Assembly%22%5D%7D"
resp2 = httpx.get(search_url)
resp2
print(search_url)

https://files.jgi.doe.gov/search/?q=1269263&a=false&h=false&d=asc&p=1&x=10&t=simple&api_version=2


In [12]:
print(json.dumps(resp.json(), indent=4))

{
    "timings": {
        "total": "0.494",
        "file_fetch": "0.023",
        "organisms": "0.455",
        "facets": "0.011",
        "count_file_total": "0.011"
    },
    "hits_total": 1450,
    "file_total": 1978,
    "unique_file_total": 1661,
    "total": 41,
    "prioritized_file_total": 1758,
    "prioritized_total": 32,
    "num_tape_files": 425,
    "score_threshold": 8.0,
    "start": 1,
    "end": 10,
    "next_page": 2,
    "cursor_id": "297b41e2d90f4eee4d2026fc9b3f7ae087429d34bb3c81fd02477c4e35646c0f:1",
    "organisms": [
        {
            "agg": "IMG_AP",
            "agg_id": 1275818,
            "kingdom": "img",
            "label": "Genome",
            "top_hit": {
                "_id": "601b71d9117e5d4b9d2c466b",
                "_highlight": null,
                "metadata": {
                    "proposal": {
                        "award_doi": "10.46936/10.25585/60001145",
                        "year": 2018,
                        "pi": {
       

In [4]:
print(json.dumps(resp2.json(), indent=4))

{
    "timings": {
        "total": "1.189",
        "file_fetch": "0.024",
        "organisms": "1.130",
        "facets": "0.032",
        "count_file_total": "0.032"
    },
    "hits_total": 68,
    "file_total": 88,
    "unique_file_total": 88,
    "total": 1,
    "prioritized_file_total": 88,
    "prioritized_total": 1,
    "num_tape_files": 61,
    "score_threshold": 8.0,
    "start": 1,
    "end": 1,
    "next_page": false,
    "cursor_id": "4413cb7a094525b23a936331f859a2c83ddde7921a91fc1c4d1fc6ecd4b08114:1",
    "organisms": [
        {
            "agg": "IMG_AP",
            "agg_id": 1269263,
            "kingdom": "img",
            "label": "Genome",
            "top_hit": {
                "_id": "5f99fe0047675a20c851c99f",
                "_highlight": null,
                "metadata": {
                    "proposal": {
                        "award_doi": "10.46936/10.25585/60001044",
                        "year": 2015,
                        "pi": {
               

In [5]:
resp = resp2.json()
len(resp["organisms"])

1

In [6]:
resp.keys()

dict_keys(['timings', 'hits_total', 'file_total', 'unique_file_total', 'total', 'prioritized_file_total', 'prioritized_total', 'num_tape_files', 'score_threshold', 'start', 'end', 'next_page', 'cursor_id', 'organisms', 'missing_organisms', 'num_missing_files', 'proposals', 'facets', 'fields', 'timeout'])

In [9]:
resp["organisms"][0].keys()

dict_keys(['agg', 'agg_id', 'kingdom', 'label', 'top_hit', 'pi', 'proposal_acceptance_date', 'work_completion_date', 'status', 'id', 'grouped_by', 'score', 'grouped_count', 'name', 'title', 'file_total', 'files', 'fileSize', 'highlight', 'pubs', 'portal_detail_id', 'data_utilization_status', 'product_search_category', 'proposal_id', 'award_doi', 'doi', 'proposal_lay_description', 'proposal_type', 'build_time'])

In [11]:
from collections import defaultdict

file_keys = defaultdict(int)
for file in resp["organisms"][0]["files"]:
    for key in file.keys():
        file_keys[key] += 1

In [12]:
file_keys

defaultdict(int,
            {'_id': 88,
             '_highlight': 88,
             'file_path': 88,
             'file_status_id': 88,
             'metadata': 88,
             'file_status': 88,
             'added_date': 88,
             'file_name': 88,
             'file_owner': 88,
             'modified_date': 88,
             'dt_to_purge': 88,
             'file_size': 88,
             'file_date': 88,
             'md5sum': 88,
             'file_type': 88,
             'file_id': 88,
             'file_group': 88,
             'user': 88,
             'file_permissions': 88,
             'data_group': 88,
             'portal_detail_id': 88,
             'dce': 12})

In [103]:
# rows with no pubmed id
no_pmid = []
# JGI Project ids
studies = defaultdict(int)
# ITS AP Ids (analysis project)
its_ap_ids = defaultdict(int)
# ITS SP IDs (sequencing project)
its_sp_ids = defaultdict(int)
# taxon_oids
tax_ids = defaultdict(int)
with open("./jgi_metadata/img_data.tsv") as infile:
    header_line = infile.readline()
    for line in infile.readlines():
        values = line.rstrip("\n\r").split("\t")
        # we want rows that: no pubmed id or genbank id, are Isolate projects, and in Permanent Draft status
        if not values[14] and not values[15] and "isolate" in values[7].lower() and "permanent draft" in values[2].lower():
            no_pmid.append(values)
            studies[values[23]] += 1
            its_ap_ids[values[24]] += 1
            its_sp_ids[values[25]] += 1
            tax_ids[values[0]] += 1

In [239]:
spid_2_row = {}
for row in no_pmid:
    spid_2_row[row[25]] = {"tax_id": row[0], "sp_name": row[4], "img_genome_id": row[6]}

In [238]:
no_pmid[0]

['2933449570',
 'Bacteria',
 'Permanent Draft',
 'Genome Sequencing for Natural Products Genomics Resource Center (NPGRC)',
 'Janibacter sp. 000076',
 'DOE Joint Genome Institute  (JGI)',
 '2933449570',
 'Genome Analysis (Isolate)',
 'Yes',
 '',
 'Yes',
 '10.46936/10.25585/60001313',
 '2019',
 'No',
 '',
 '',
 '',
 'Illumina NovaSeq S4',
 'Level 1: Standard Draft',
 'Complete',
 'Whole Genome Sequencing',
 '3378837',
 '3406',
 '0',
 '1301683',
 '1292578',
 '506106']

In [43]:
header = header_line.strip("\n\r").split("\t")
for idx, txt in enumerate(header):
    print(f"{idx}\t{txt}")

0	taxon_oid
1	NCBI Domain
2	Sequencing Status
3	Study Name
4	Genome Name / Sample Name
5	Sequencing Center
6	IMG Genome ID
7	GOLD Analysis Project Type
8	High Quality
9	Has Coverage
10	Is Public
11	Award DOI
12	Funding Year
13	Is Published
14	Pubmed ID
15	NCBI GenBank ID
16	Sequencing Depth
17	Sequencing Method
18	Sequencing Quality
19	Sequencing Status
20	Sequencing Strategy
21	Genome Size  * assembled
22	Gene Count  * assembled
23	JGI Project ID / ITS SP ID
24	ITS AP ID
25	ITS SP ID
26	ITS Proposal ID


In [156]:
len(no_pmid)

4556

In [105]:
len(studies)

310

In [157]:
len(its_sp_ids)

4533

In [68]:
no_pmid[0]

['2933449570',
 'Bacteria',
 'Permanent Draft',
 'Genome Sequencing for Natural Products Genomics Resource Center (NPGRC)',
 'Janibacter sp. 000076',
 'DOE Joint Genome Institute  (JGI)',
 '2933449570',
 'Genome Analysis (Isolate)',
 'Yes',
 '',
 'Yes',
 '10.46936/10.25585/60001313',
 '2019',
 'No',
 '',
 '',
 '',
 'Illumina NovaSeq S4',
 'Level 1: Standard Draft',
 'Complete',
 'Whole Genome Sequencing',
 '3378837',
 '3406',
 '0',
 '1301683',
 '1292578',
 '506106']

In [78]:
# some don't have SP ids, some SP ids are duplicated. Skip these.
sorted(its_sp_ids.items(), key=lambda x: x[1], reverse=True)

[('', 18),
 ('1020215', 2),
 ('1269118', 2),
 ('1284500', 2),
 ('1139853', 2),
 ('1147427', 2),
 ('1020212', 2),
 ('1292578', 1),
 ('1352244', 1),
 ('1025314', 1),
 ('1352100', 1),
 ('1340064', 1),
 ('1352570', 1),
 ('1340081', 1),
 ('1352135', 1),
 ('1551573', 1),
 ('1401606', 1),
 ('1401421', 1),
 ('1306885', 1),
 ('1392001', 1),
 ('1214814', 1),
 ('1357722', 1),
 ('1413023', 1),
 ('1292638', 1),
 ('1030956', 1),
 ('1381299', 1),
 ('1381348', 1),
 ('1413639', 1),
 ('1357681', 1),
 ('1292660', 1),
 ('1357828', 1),
 ('1340402', 1),
 ('1551314', 1),
 ('1347379', 1),
 ('1551413', 1),
 ('1401642', 1),
 ('1219219', 1),
 ('1385549', 1),
 ('1214796', 1),
 ('1351990', 1),
 ('1317087', 1),
 ('1352445', 1),
 ('1355818', 1),
 ('1360769', 1),
 ('1401521', 1),
 ('1357648', 1),
 ('1352535', 1),
 ('1307022', 1),
 ('1401783', 1),
 ('1385585', 1),
 ('1413440', 1),
 ('1413358', 1),
 ('1361092', 1),
 ('1381315', 1),
 ('1401476', 1),
 ('1340347', 1),
 ('1361132', 1),
 ('1340214', 1),
 ('1381045', 1),
 ('

In [87]:
singleton_its_sps = [i[0] for i in sorted(its_sp_ids.items(), key=lambda x: x[1], reverse=True)[7:]]
singleton_its_sps[1]

'1352244'

In [110]:
len(singleton_its_sps)

4526

In [107]:
def query_maker(its_sp_id: str) -> str:
    base_url = "https://files.jgi.doe.gov"
    return base_url + f"/img_file_list/?its_sp_id={its_sp_id}&api_version=2&a=false&h=false&d=asc&p=1&x=10&t=simple&ff[file_type]=fastq"

def run_query(its_sp_id: str) -> dict:
    resp = httpx.get(query_maker(its_sp_id))
    return resp.json()

In [108]:
result = run_query(singleton_its_sps[1])

In [112]:
# start with just the first, say, 100
spids_of_interest = singleton_its_sps[:100]

In [116]:
import time
for spid in spids_of_interest:
    meta_result = run_query(spid)
    with open(f"./jgi_metadata/{spid}_metadata.json", "w") as out_meta:
        json.dump(meta_result, out_meta, indent=4)

In [109]:
print(json.dumps(result["organisms"][0]["files"], indent=4))
fids = []
for res_file in result["organisms"][0]["files"]:
    print(f'{result["organisms"][0]["agg"]}-{result["organisms"][0]["agg_id"]}')
    print(f'{res_file["metadata"]["sequencing_project_id"]}\t{res_file["file_status"]}\t{res_file["file_name"]}\t{res_file["_id"]}')
    fids.append(res_file["_id"])
file_request_packet = {
    "ids": {
        f'{result["organisms"][0]["agg"]}-{result["organisms"][0]["agg_id"]}': {
            "file_ids": fids
        }
    }
}
print(file_request_packet)                

[
    {
        "_id": "622f645fe99caa81935b477e",
        "_highlight": null,
        "file_path": "/global/dna/dm_archive/sdm/illumina/05/26/64",
        "file_status_id": 13,
        "metadata": {
            "proposal": {
                "award_doi": "10.46936/10.25585/60001355",
                "pi": {
                    "country": "United States",
                    "institution": "The Scripps Research Institute - Florida",
                    "email_address": "shenb@scripps.edu",
                    "last_name": "Shen",
                    "middle_name": "",
                    "contact_id": 23529,
                    "first_name": "Ben"
                },
                "lay_description": "The Natural Products Genomics Resource Center (NPGRC), a collaborative effort among the Joint Genome Institute (JGI), The Scripps Research Institute (TSRI), and University of Minnesota (UMN), will be established as a genomic data resource for the broader scientific community to enable natu

In [145]:
# some simplifying housekeeping
# map file names -> img_sp_ids
file_2_spid = {}
file_2_filename = {}
filename_2_spid = {}
# map spid to file(s)
spid_2_files = {}
file_data = []

file_request_query = {"ids": {}}
for spid in spids_of_interest[:100]:
    print(spid)
    with open(f"./jgi_metadata/{spid}_metadata.json", "r") as in_meta:
        meta = json.load(in_meta)
        if "organisms" not in meta:
            print(f"{spid} no organisms key found")
            continue
        if (len(meta["organisms"]) == 0):
            print(f"{spid} no organisms found")
            continue
        if "files" not in meta["organisms"][0] or len(meta["organisms"][0]["files"]) == 0:
            print(f"{spid} no fastq files found")
            continue
        agg_id = (f'{meta["organisms"][0]["agg"]}-{meta["organisms"][0]["agg_id"]}')
        file_ids = []
        for data_file in meta["organisms"][0]["files"]:
            file_ids.append(data_file["_id"])
            file_2_filename[data_file["_id"]] = data_file["file_name"]
            filename_2_spid[data_file["file_name"]] = spid
            file_2_spid[data_file["_id"]] = spid
        file_request_query["ids"][agg_id] = {"file_ids": file_ids}
        spid_2_files[spid] = file_ids
        file_data.append(data_file)

1292578
1352244
1025314
1352100
1340064
1352570
1340081
1352135
1551573
1401606
1401421
1306885
1392001
1214814
1357722
1413023
1292638
1030956
1381299
1381348
1413639
1357681
1292660
1357828
1340402
1551314
1347379
1551413
1401642
1219219
1385549
1214796
1351990
1317087
1352445
1355818
1360769
1401521
1357648
1352535
1307022
1401783
1385585
1413440
1413358
1361092
1381315
1401476
1340347
1361132
1340214
1381045
1413287
1551102
1018461
1018461 no organisms found
1413237
1551217
1270347
1317170
1392030
1352195
1352245
1413662
1551532
1551017
1307026
1357514
1306932
1292629
1381670
1392052
1392148
1340084
1352183
1134423
1551412
1357471
1352097
1392023
1352571
1407839
1401299
1551334
1381444
1551451
1381328
1139671
1155679
1551151
1357731
1551015
1551289
1105609
1352409
1401552
1401352
1214714
1586655
1269079
1340370


In [134]:
file_request_query

{'ids': {'IMG_SP-1292578': {'file_ids': ['6086dda253a8c649b0cb36b8']},
  'IMG_SP-1352244': {'file_ids': ['622f645fe99caa81935b477e']},
  'IMG_SP-1025314': {'file_ids': ['529679d4067c0121bf0d6373',
    '52964be3067c0121bf0d621e']},
  'IMG_SP-1352100': {'file_ids': ['62029526f9c444ffdffcb7e0']},
  'IMG_SP-1340064': {'file_ids': ['62029171f9c444ffdffcb74a']},
  'IMG_SP-1352570': {'file_ids': ['6228c68de99caa81935ae628']},
  'IMG_SP-1340081': {'file_ids': ['620290c8f9c444ffdffcb730']},
  'IMG_SP-1352135': {'file_ids': ['62029384f9c444ffdffcb7a1']},
  'IMG_SP-1551573': {'file_ids': ['6835caf1e69ed90826268275']},
  'IMG_SP-1401606': {'file_ids': ['63a8dcc63b5d0133c7411b0c']},
  'IMG_SP-1401421': {'file_ids': ['63a8dc6a3b5d0133c7411a11']},
  'IMG_SP-1306885': {'file_ids': ['6099791a53a8c649b0cc08a3']},
  'IMG_SP-1392001': {'file_ids': ['636eb350be000c239f764c54']},
  'IMG_SP-1214814': {'file_ids': ['5c13125446d1e64a530fbfca']},
  'IMG_SP-1357722': {'file_ids': ['6269e082f21e5a14d08da48b']},
 

In [146]:
sso_token = "Bearer /api/sessions/3360f5eec4b04788500b6b937e8e6fdb"

In [143]:
filename_2_spid

{'52508.4.363682.GCCTTGTT-GCCTTGTT.fastq.gz': '1292578',
 '52664.1.415157.TTGGACGT-TTGGACGT.fastq.gz': '1352244',
 '7479.7.74223.AGTTCC.adnq.fastq.gz': '1025314',
 '7479.7.74223.AGTTCC.fastq.gz': '1025314',
 '52647.2.412368.AGTTACGG-AGTTACGG.fastq.gz': '1352100',
 '52647.2.412368.CACAAGTC-CACAAGTC.fastq.gz': '1340064',
 '52659.4.414109.CCAGTGTT-CCAGTGTT.fastq.gz': '1352570',
 '52647.2.412368.TTAGGTCG-TTAGGTCG.fastq.gz': '1340081',
 '52647.2.412368.TTGATCCG-TTGATCCG.fastq.gz': '1352135',
 '53127.1.597830.TGAGGTAAGA-CTCTGCAAGC.fastq.gz': '1551573',
 '52789.4.453124.TATCGGTC-TATCGGTC.fastq.gz': '1401606',
 '52789.3.453034.TATGGCAG-TATGGCAG.fastq.gz': '1401421',
 '52512.4.365108.TCATCACC-TCATCACC.fastq.gz': '1306885',
 'pbio-2705.26963.bc1011_BAK8A_OA--bc1011_BAK8A_OA.ccs.filter.fastq.gz': '1392001',
 '12774.2.284685.GAGGACTT-AAGTCCTC.fastq.gz': '1214814',
 '52682.2.418228.TCCGTATG-TCCGTATG.fastq.gz': '1357722',
 '52807.3.457668.TTGTGTGC-TTGTGTGC.fastq.gz': '1413023',
 '52508.4.363682.GTCT

In [129]:
resp = httpx.post("https://files.jgi.doe.gov/request_archived_files/", json=file_request_query, headers={"Authorization": sso_token})

In [130]:
resp.json()

{'updated_count': 1,
 'restored_count': 10,
 'request_id': 525995,
 'request_status_url': 'https://files.jgi.doe.gov/request_archived_files/requests/525995'}

In [131]:
status_resp = httpx.post("https://files.jgi.doe.gov/request_archived_files/requests/", json={"request_ids": [525995]}, headers={"Authorization": sso_token})

In [132]:
status_resp.json()

{'525995': {'file_ids': ['6835caf1e69ed90826268275',
   '62029384f9c444ffdffcb7a1',
   '6228c68de99caa81935ae628',
   '62029171f9c444ffdffcb74a',
   '620290c8f9c444ffdffcb730',
   '62029526f9c444ffdffcb7e0',
   '63a8dcc63b5d0133c7411b0c',
   '6086dda253a8c649b0cb36b8',
   '622f645fe99caa81935b477e',
   '529679d4067c0121bf0d6373',
   '52964be3067c0121bf0d621e'],
  'status': 'pending',
  'expiration_date': '2025-12-18T10:34:07.121276-08:00'}}

In [153]:
from pathlib import Path

def build_instructions(file_data: list[dict]) -> dict:
    obj_set = []

    for file_info in file_data:
        file_path = Path(file_info["file_path"].split("dm_archive")[-1]) / file_info["file_name"]
        obj_name = file_info["file_name"] + "_reads"
        obj_set.append({
            "data_type": "fastq_reads_interleaved",
            "parameters": {            
                "fastq_fwd_staging_file_name": str(file_path),
                "name": obj_name,
                "sequencing_tech": "Unknown",
                "single_genome": 1,
                "read_orientation_outward": 0,
                "insert_size_std_dev": None,
                "insert_size_mean": None
            }
        })
    
    return {
        "protocol": "KBase narrative import",
        "objects": obj_set
    }

In [154]:
inst = build_instructions(file_data)

In [155]:
inst

{'protocol': 'KBase narrative import',
 'objects': [{'data_type': 'fastq_reads_interleaved',
   'parameters': {'fastq_fwd_staging_file_name': '/sdm/illumina/05/25/08/52508.4.363682.GCCTTGTT-GCCTTGTT.fastq.gz',
    'name': '52508.4.363682.GCCTTGTT-GCCTTGTT.fastq.gz_reads',
    'sequencing_tech': 'Unknown',
    'single_genome': 1,
    'read_orientation_outward': 0,
    'insert_size_std_dev': None,
    'insert_size_mean': None}},
  {'data_type': 'fastq_reads_interleaved',
   'parameters': {'fastq_fwd_staging_file_name': '/sdm/illumina/05/26/64/52664.1.415157.TTGGACGT-TTGGACGT.fastq.gz',
    'name': '52664.1.415157.TTGGACGT-TTGGACGT.fastq.gz_reads',
    'sequencing_tech': 'Unknown',
    'single_genome': 1,
    'read_orientation_outward': 0,
    'insert_size_std_dev': None,
    'insert_size_mean': None}},
  {'data_type': 'fastq_reads_interleaved',
   'parameters': {'fastq_fwd_staging_file_name': '/sdm/illumina/00/74/79/7479.7.74223.AGTTCC.fastq.gz',
    'name': '7479.7.74223.AGTTCC.fastq.gz

In [152]:
list(file_2_spid.keys())

['6086dda253a8c649b0cb36b8',
 '622f645fe99caa81935b477e',
 '529679d4067c0121bf0d6373',
 '52964be3067c0121bf0d621e',
 '62029526f9c444ffdffcb7e0',
 '62029171f9c444ffdffcb74a',
 '6228c68de99caa81935ae628',
 '620290c8f9c444ffdffcb730',
 '62029384f9c444ffdffcb7a1',
 '6835caf1e69ed90826268275',
 '63a8dcc63b5d0133c7411b0c',
 '63a8dc6a3b5d0133c7411a11',
 '6099791a53a8c649b0cc08a3',
 '636eb350be000c239f764c54',
 '5c13125446d1e64a530fbfca',
 '6269e082f21e5a14d08da48b',
 '63f8d6eade99bdbaa7ef2cf5',
 '6086de9853a8c649b0cb36f0',
 '5329394749607a1be00599ee',
 '53297fef49607a1be0059a93',
 '63039c791a1c5f71256c5c1a',
 '63039c921a1c5f71256c5c68',
 '63f8d7f2de99bdbaa7ef2f51',
 '6269e17cf21e5a14d08da4bd',
 '6086deed53a8c649b0cb3704',
 '625487e7f21e5a14d08cb997',
 '61e8cd7cb0df7a8c0db819e5',
 '684b0e1589cc9f214d4f2f09',
 '6247812fc6477704c332f18d',
 '6835caf3e69ed9082626827b',
 '63a8dce93b5d0133c7411b33',
 '5bc913be46d1e64cf8dace18',
 '5bc8cd4e46d1e64cf8dacb4c',
 '63ea5cfcd22e9615ded867f2',
 '5c1311df46d1

Variable              Type             Data/Info
------------------------------------------------
Path                  type             <class 'pathlib.Path'>
agg_id                str              IMG_SP-1340370
body                  list             n=15587
build_instructions    function         <function build_instructions at 0x1102b4d60>
data_file             dict             n=21
defaultdict           type             <class 'collections.defaultdict'>
fids                  list             n=1
file                  dict             n=21
file_2_filename       dict             n=107
file_2_spid           dict             n=107
file_data             list             n=99
file_id               str              61e8ce15b0df7a8c0db81a10
file_ids              list             n=1
file_keys             defaultdict      defaultdict(<class 'int'><...>tail_id': 88, 'dce': 12})
file_request_packet   dict             n=1
file_request_query    dict             n=1
filename_2_spid       dict   

In [175]:
num_unpurged_files = 10
total = len(singleton_its_sps[2216:])
current = 0
for idx, spid in enumerate(singleton_its_sps[2216:]):
    print(f"{idx}/{total} {spid}")
    meta_result = run_query(spid)
    # find some metadata that hasn't been purged. Just a few. Please....
    if len(meta_result.get("organisms", [])) and len(meta_result["organisms"][0].get("files", [])):
        for file_info in meta_result["organisms"][0]["files"]:
            is_purged = file_info["file_status"].lower() == "purged"
            if not is_purged:
                print(f"\tGOT ONE! {spid}")
                spids_to_fetch.append(spid)
                with open(f"./jgi_metadata/{spid}_metadata.json", "w") as out_meta:
                    json.dump(meta_result, out_meta, indent=4)
    if len(spids_to_fetch) >= num_unpurged_files:
        break
    if idx % 100 == 0:
        print("taking a break for 5 seconds")
        time.sleep(5)
        print("break's over!")

0/2310 1306926
taking a break for 5 seconds
break's over!
1/2310 1134408
2/2310 1360795
3/2310 1413781
4/2310 1381698
5/2310 1551231
6/2310 1134407
7/2310 1357517
8/2310 1139666
9/2310 1292635
10/2310 1392081
11/2310 1392117
12/2310 1392109
13/2310 1381511
14/2310 1352387
15/2310 1401261
16/2310 1269061
17/2310 1551274
18/2310 1214731
19/2310 1413585
20/2310 1413602
21/2310 1340198
22/2310 1317446
23/2310 1360997
24/2310 1381058
25/2310 1361088
26/2310 1551395
27/2310 1351885
28/2310 1347388
29/2310 1381458
30/2310 1306997
31/2310 1401329
32/2310 1412945
33/2310 1352417
34/2310 1361086
35/2310 1282091
36/2310 1413044
37/2310 1360765
38/2310 1413452
39/2310 1326024
40/2310 1317117
41/2310 1401908
42/2310 1381355
43/2310 1550951
44/2310 1352273
45/2310 1214693
46/2310 1360992
47/2310 1401300
48/2310 1352382
49/2310 1214775
50/2310 1326097
51/2310 1381443
52/2310 1219235
53/2310 1551467
54/2310 1413000
55/2310 1340268
56/2310 1134414
57/2310 1551501
58/2310 1352397
59/2310 1360762
60/2310

In [179]:
spids_to_fetch

['1248680',
 '1030857',
 '1030857',
 '1053055',
 '1227824',
 '1248546',
 '1347351',
 '1186026',
 '1147911',
 '1186048']

In [181]:
revised_spids_to_fetch = spids_to_fetch[0:1] + spids_to_fetch[2:]
revised_spids_to_fetch

['1248680',
 '1030857',
 '1053055',
 '1227824',
 '1248546',
 '1347351',
 '1186026',
 '1147911',
 '1186048']

In [190]:
# some simplifying housekeeping
# map file names -> img_sp_ids
new_file_2_spid = {}
new_file_2_filename = {}
new_filename_2_spid = {}
# map spid to file(s)
new_spid_2_files = {}
new_file_data = []

new_file_request_query = {"ids": {}}
for spid in revised_spids_to_fetch:
    print(spid)
    with open(f"./jgi_metadata/{spid}_metadata.json", "r") as in_meta:
        meta = json.load(in_meta)
        if "organisms" not in meta:
            print(f"{spid} no organisms key found")
            continue
        if (len(meta["organisms"]) == 0):
            print(f"{spid} no organisms found")
            continue
        if "files" not in meta["organisms"][0] or len(meta["organisms"][0]["files"]) == 0:
            print(f"{spid} no fastq files found")
            continue
        agg_id = (f'{meta["organisms"][0]["agg"]}-{meta["organisms"][0]["agg_id"]}')
        file_ids = []
        for data_file in meta["organisms"][0]["files"]:
            print(f"{data_file['_id']} - {data_file['file_status']}")
            if data_file["file_status"] != "RESTORED":
                continue
            file_ids.append(data_file["_id"])
            new_file_2_filename[data_file["_id"]] = data_file["file_name"]
            new_filename_2_spid[data_file["file_name"]] = spid
            new_file_2_spid[data_file["_id"]] = spid
            new_file_data.append(data_file)
        new_file_request_query["ids"][agg_id] = {"file_ids": file_ids}
        new_spid_2_files[spid] = file_ids


1248680
67ae064768dd0c5de8e12ada - RESTORED
1030857
5329352d49607a1be00599a1 - RESTORED
5329760c49607a1be0059a5c - PURGED
1053055
545d865e0d87855284890b94 - PURGED
545d5e010d87855284890b40 - RESTORED
1227824
67c631d367ef7b237e943332 - BACKUP_COMPLETE
1248546
67d0973ed72b25923552b58d - RESTORED
1347351
6267f6a1f21e5a14d08d81ce - RESTORED
6268352cf21e5a14d08d89f5 - PURGED
1186026
67c6318d67ef7b237e9432a9 - BACKUP_COMPLETE
1147911
67c6318c67ef7b237e94329d - BACKUP_COMPLETE
1186048
67d0972fd72b25923552b524 - RESTORED


In [191]:
new_file_request_query

{'ids': {'IMG_SP-1248680': {'file_ids': ['67ae064768dd0c5de8e12ada']},
  'IMG_SP-1030857': {'file_ids': ['5329352d49607a1be00599a1']},
  'IMG_SP-1053055': {'file_ids': ['545d5e010d87855284890b40']},
  'IMG_SP-1227824': {'file_ids': []},
  'IMG_SP-1248546': {'file_ids': ['67d0973ed72b25923552b58d']},
  'IMG_SP-1347351': {'file_ids': ['6267f6a1f21e5a14d08d81ce']},
  'IMG_SP-1186026': {'file_ids': []},
  'IMG_SP-1147911': {'file_ids': []},
  'IMG_SP-1186048': {'file_ids': ['67d0972fd72b25923552b524']}}}

In [194]:
new_file_2_spid.keys()

dict_keys(['67ae064768dd0c5de8e12ada', '5329352d49607a1be00599a1', '545d5e010d87855284890b40', '67d0973ed72b25923552b58d', '6267f6a1f21e5a14d08d81ce', '67d0972fd72b25923552b524'])

In [193]:
build_instructions(new_file_data)

{'protocol': 'KBase narrative import',
 'objects': [{'data_type': 'fastq_reads_interleaved',
   'parameters': {'fastq_fwd_staging_file_name': '/sdm/illumina/05/30/90/53090.2.581637.CCACTCGAGC-AGGACTCTTC.fastq.gz',
    'name': '53090.2.581637.CCACTCGAGC-AGGACTCTTC.fastq.gz_reads',
    'sequencing_tech': 'Unknown',
    'single_genome': 1,
    'read_orientation_outward': 0,
    'insert_size_std_dev': None,
    'insert_size_mean': None}},
  {'data_type': 'fastq_reads_interleaved',
   'parameters': {'fastq_fwd_staging_file_name': '/sdm/illumina/00/77/79/7779.3.83550.AAGCGA.fastq.gz',
    'name': '7779.3.83550.AAGCGA.fastq.gz_reads',
    'sequencing_tech': 'Unknown',
    'single_genome': 1,
    'read_orientation_outward': 0,
    'insert_size_std_dev': None,
    'insert_size_mean': None}},
  {'data_type': 'fastq_reads_interleaved',
   'parameters': {'fastq_fwd_staging_file_name': '/sdm/illumina/00/84/65/8465.8.102013.GACGAC.fastq.gz',
    'name': '8465.8.102013.GACGAC.fastq.gz_reads',
    'se

In [195]:
# Need to make a table - should have
# * spid x
# * taxon_oid 
# * species name
# * file name
# * original UPA
# * narrative it was run in
# * predicted species

How to map:  
Not all files are used or downloaded, not all spids were used.  
Can start with UPA and filename?

Run list_objects on ws 239038? = get UPA, get object name. strip "_reads" off name = file name

We have file_data and new_file_data (merge them!)  
this gets us file name and id (file_name, _id)

we have file_2_spid and new file_2_spid (merge those too!) - gets us the spid  
spid gets us metadata, so we get taxon oid, name from img_data.tsv file

Just need the narrative id - can parse out of logs  
And predicted spp from final run... we'll get there next. Maybe LLM help.

In [213]:
import os
from narrative_llm_agent.kbase.clients.workspace import Workspace
ws = Workspace(token="NLFMP7ZOEJCLXRB6FYV6RNJH35XEOYN3")

In [205]:
ws._endpoint

'https://kbase.us/services/ws'

In [214]:
reads_objs = ws.list_workspace_objects(239038, as_dict=True)

In [224]:
reads_objs[3]

{'ws_id': 239038,
 'obj_id': 5,
 'version': 1,
 'name': '52659.4.414109.CCAGTGTT-CCAGTGTT.fastq.gz_reads',
 'ws_name': 'wjriehl:narrative_1764881155555',
 'type': 'KBaseFile.PairedEndLibrary-2.1',
 'saved': '2025-12-08T23:06:11+0000',
 'saved_by': 'wjriehl',
 'size_bytes': 760,
 'metadata': None,
 'path': [],
 'upa': '239038/5/1'}

In [226]:
filename_2_spid[reads_objs[2]["name"].rstrip("_reads")]

'1551573'

In [229]:
all_file_data = file_data + new_file_data

In [246]:
print(json.dumps(next(f for f in all_file_data if f["file_name"] == reads_objs[2]["name"].rstrip("_reads")), indent=4))

{
    "_id": "6835caf1e69ed90826268275",
    "_highlight": null,
    "file_path": "/global/dna/dm_archive/sdm/illumina/05/31/27",
    "file_status_id": 10,
    "metadata": {
        "proposal": {
            "award_doi": "10.46936/10.25585/60008892",
            "pi": {
                "institution": "Univ of North Dakota",
                "country": "United States",
                "email_address": "kjvenkat1955@gmail.com",
                "last_name": "Venkateswaran",
                "contact_id": 50073,
                "middle_name": "",
                "first_name": "Kasthuri"
            },
            "lay_description": "The Biotechnology and Planetary Protection Group at Jet Propulsion Laboratory (JPL) has been at the forefront of a significant effort focused on studying bacterial and fungal strains from both the International Space Station (ISS) and Mars mission assembly cleanrooms. Over the course of six years and spanning six different missions to the ISS, as well as approxim

In [290]:
# final_table_by_upa = {}
for obj in reads_objs[2:]:
    upa = obj["upa"]
    obj_name = obj["name"]
    file_name = obj["name"].rstrip("_reads")
    print(file_name)
    file_info = next((f for f in all_file_data if f["file_name"] == file_name), {})
    print(file_info)
    spid = str(file_info["metadata"]["sequencing_project_id"])
    if spid.startswith("[") and spid.endswith("]"):
        spid = spid[1:-1]
    print(spid)
    meta_row = spid_2_row[spid]
    final_table_by_upa[upa] = {
        "upa": upa,
        "obj_name": obj_name,
        "file_name": file_name,
        "spid": spid,
        "file_id": file_info["_id"],
    } | meta_row

53127.1.597830.TGAGGTAAGA-CTCTGCAAGC.fastq.gz
{'_id': '6835caf1e69ed90826268275', '_highlight': None, 'file_path': '/global/dna/dm_archive/sdm/illumina/05/31/27', 'file_status_id': 10, 'metadata': {'proposal': {'award_doi': '10.46936/10.25585/60008892', 'pi': {'institution': 'Univ of North Dakota', 'country': 'United States', 'email_address': 'kjvenkat1955@gmail.com', 'last_name': 'Venkateswaran', 'contact_id': 50073, 'middle_name': '', 'first_name': 'Kasthuri'}, 'lay_description': 'The Biotechnology and Planetary Protection Group at Jet Propulsion Laboratory (JPL) has been at the forefront of a significant effort focused on studying bacterial and fungal strains from both the International Space Station (ISS) and Mars mission assembly cleanrooms. Over the course of six years and spanning six different missions to the ISS, as well as approximately 20 years for Mars mission cleanrooms, they have successfully isolated and sequenced 3,500 strains.\n\nIn the next phase of this project, we a

In [294]:
# Now, get the narrative id for each one
# start with getting log files
upa_to_logfile = {}
logfiles = list(filter(lambda f: f.startswith("llm_genome_annotation_239038"), os.listdir("../")))

for upa in final_table_by_upa.keys():
    logfile = next(f for f in logfiles if upa.replace("/", "_") in f)
    upa_to_logfile[upa] = logfile

for upa, logfile in upa_to_logfile.items():
    with open(Path("..") / logfile) as log:
        for line in log.readlines():
            if "created narrative with id" in line:
                narrative_id = line.rstrip().split()[-1]
                print(f"{upa}\t{narrative_id}")
                final_table_by_upa[upa]["narrative_id"] = narrative_id
                final_table_by_upa[upa]["narrative_url"] = f"https://narrative.kbase.us/narrative/{narrative_id}"

239038/4/1	239697
239038/5/1	239700
239038/7/1	239694
239038/8/1	239691
239038/11/1	239696
239038/12/1	239698
239038/13/1	239699
239038/14/1	239692
239038/15/1	239693
239038/20/1	239738
239038/21/1	239740
239038/23/1	239736
239038/28/1	239739
239038/30/1	239741
239038/31/1	239734
239038/32/1	239742
239038/33/1	239735
239038/34/1	239733
239038/38/1	239737
239038/41/1	239789
239038/42/1	239792
239038/43/1	239786
239038/45/1	239752
239038/46/1	239750
239038/47/1	239751
239038/48/1	239746
239038/53/1	239787
239038/55/1	239748
239038/56/1	239753
239038/60/1	239791
239038/62/1	239795
239038/64/1	239793
239038/68/1	239788
239038/69/1	239794
239038/70/1	239790
239038/74/1	239850
239038/75/1	239858
239038/76/1	239855
239038/78/1	239851
239038/79/1	239849
239038/80/1	239854
239038/85/1	239853
239038/87/1	239857
239038/89/1	239852
239038/90/1	239856
239038/93/1	239873
239038/94/1	239865
239038/95/1	239869
239038/96/1	239867
239038/98/1	239872
239038/101/1	239868
239038/103/1	239866
239038/108/1	2

In [297]:
cols = [
    "JGI sequencing project id",
    "IMG genome id",
    "File id",
    "File name",
    "Original upa",
    "Generated narrative id",
    "Narrative url",
    "Species name",
]

file_lines = ["\t".join(cols)]
for data in final_table_by_upa.values():
    if "narrative_id" not in data:
        print(data)
    data_row = [
        data["spid"],
        data["img_genome_id"],
        data["file_id"],
        data["file_name"],
        data["upa"],
        data["narrative_id"],
        data["narrative_url"],
        data["sp_name"]
    ]
    file_lines.append("\t".join(data_row))

with open("img_llm_annotations.tsv", "w") as outfile:
    outfile.write("\n".join(file_lines))

In [304]:
# make all those narratives public
for data in final_table_by_upa.values():
    ws.make_kbase_jsonrpc_1_call("Workspace.set_global_permission", [{"id": data["narrative_id"], "new_permission": "r"}])